In [29]:
import pandas as pd
import ast  # For safely evaluating string-formatted lists
import plotly.graph_objects as go

arenas_df = pd.read_csv("../#2 Data Storage/Utils/arenas.csv")
arenas_dict = {}

for row in arenas_df.itertuples():
    arenas_dict[row.Arena_ID] = row.Arena_Name

def check_for_card(card_list_str, card_name):
    """
    Safely parses a string representation of a card list
    and checks if a specific card is present.
    """
    try:
        card_list = ast.literal_eval(card_list_str)
        if not isinstance(card_list, list):
            return False
        for card_tuple in card_list:
            if isinstance(card_tuple, tuple) and len(card_tuple) > 0 and card_tuple[0] == card_name:
                return True
    except (ValueError, SyntaxError, TypeError):
        return False
    return False

def arena_win_loss_data_collection():
    try:
        # --- Load Troop Names ---
        troop_file_name = '../#2 Data Storage/Utils/troop_name.csv' 
        troop_df = pd.read_csv(troop_file_name)
        troop_list = troop_df['Troop_name'].unique()
        print(f"Loaded {len(troop_list)} unique troop names.")
        # Create a set for fast lookup
        troop_set = set(troop_list) 
        
        # --- Load Battle Log Data ---
        file_name = '../#2 Data Storage/Processed Data/preprocessed_battle_log_full_batch.csv' 
        df = pd.read_csv(file_name)
        print("Data loaded. Processing for all troops...")

        # --- Process data for ALL troops ---
        card_data = [] # Renamed from all_card_data to match original

        for row in df.itertuples(index=False):
            # Check player 0's cards
            try:
                player_0_cards = {card[0] for card in ast.literal_eval(row.players_0_spells) if isinstance(card, tuple) and len(card) > 0}
                player_0_cards_evo = {card[2] for card in ast.literal_eval(row.players_0_spells) if isinstance(card, tuple) and len(card) > 2}
            except (ValueError, SyntaxError, TypeError):
                player_0_cards = set()
                
            # Check player 1's cards
            try:
                player_1_cards = {card[0] for card in ast.literal_eval(row.players_1_spells) if isinstance(card, tuple) and len(card) > 0}
                player_1_cards_evo = {card[2] for card in ast.literal_eval(row.players_1_spells) if isinstance(card, tuple) and len(card) > 2}
            except (ValueError, SyntaxError, TypeError):
                player_1_cards = set()

            # Find which troops from our list were used
            player_0_troops_used = player_0_cards.intersection(troop_set)
            player_1_troops_used = player_1_cards.intersection(troop_set)

            # Log data for player 0
            for troop_name, evo_bin in zip(player_0_troops_used, player_0_cards_evo):
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_0_winner == 1 else 'Lost',
                    'card_name': troop_name, # Add the card_name column
                    'evo': evo_bin
                })
                
            # Log data for player 1
            for troop_name, evo_bin in zip(player_1_troops_used, player_1_cards_evo):
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_1_winner == 1 else 'Lost',
                    'card_name': troop_name, # Add the card_name column
                    'evo': evo_bin
                })

        # --- Create and Group DataFrame ---
        if not card_data:
            print(f"No usage data found for any troops.")
        else:
            card_df = pd.DataFrame(card_data)
            
            # Group data by arena, card_name, and outcome to get counts
            # This is the key change to support the dropdown
            grouped_df = card_df.groupby(['arena', 'card_name', 'outcome', 'evo']).size().reset_index(name='count')
            grouped_df['arena'] = grouped_df['arena'].map(arenas_dict)  # Map arena IDs to names
            grouped_df = grouped_df.dropna(subset=['arena'])

            grouped_df.to_csv("../#2 Data Storage/Visualized Data/arenawise_card_win_loss.csv")
            
    except FileNotFoundError as e:
        print(f"Error: A required file was not found. {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- Run the function ---
arena_win_loss_data_collection()

Loaded 121 unique troop names.
Data loaded. Processing for all troops...
